In [75]:
import numpy as np
import os
import random
import imageio
import argparse
import torch
import torch.nn as nn
import torch.utils.data as da

# 1. 读取数据，构建训练集和测试集

In [13]:
def get_images(paths, labels, nb_samples = None, shuffle = True):
    """
    获取一组字符文件夹和标签，并返回带有标签的图像文件的路径。
    输入:
        paths: 一个字符文件夹的列表
        labels: 与路径长度相同的列表或numpy数组
        nb_samples: 每个字符检索的图像数量
    输出:
        由元组(标签, 图像路径)构成的列表
    """
    if nb_samples is not None:
        sampler = lambda x: random.sample(x, nb_samples)
    else:
        sampler = lambda x: x
    images_labels = [(i, os.path.join(path, image))
                     for i, path in zip(labels, paths)
                     for image in sampler([pathstr for pathstr in os.listdir(path) if pathstr[-4:] == '.png' ])]
    if shuffle:
        random.shuffle(images_labels)
    return images_labels

In [39]:
def image_file_to_array(filename, dim_input):
    """
    读取图像路径并返回numpy数组
    输入:
        filename: 图像文件名称
        dim_input: 图像的扁平形状
    输出:
        单通道图像
    """
    image = imageio.v2.imread(filename)
    image = image.reshape([dim_input])
    image = image.astype(np.float32) / 255.0
    image = 1.0 - image

    return image

In [40]:
def pair_shuffle(array_a, array_b):
    """
    获取一个图像数组和一个标签数组
    输出:
        打乱的图像数组和标签数组
    """
    temp_perm = np.random.permutation(array_a.shape[0])
    array_a = array_a[temp_perm]
    array_b = array_b[temp_perm]
    return array_a, array_b

In [41]:
def LoadData(num_classes = 50, num_samples_per_class_train = 15, num_samples_per_class_test = 5, seed = 1):
    """
    加载数据并将其分割为训练和测试集
    输入:
        num_classes: 采用的类数，-1表示使用所有类
        num_samples_per_class_train: 每个类用于训练的样本数量
        num_samples_per_class_test: 每个类用于测试的样本数量
        seed: 随机种子以确保结果一致
    输出:
        一个元组：(1)用于训练的图像(2)用于训练的标签(3)用于测试的图像，以及(4)用于测试的标签
            (1) 形状[num_classes * num_samples_per_class_train, 784]，二进制像素的Numpy数组
            (2) 形状[num_classes * num_samples_per_class_train]的Numpy数组，类标签的整数
            (3) 形状[num_classes * num_samples_per_class_test, 784]，二进制像素的Numpy数组
            (4) 形状[num_classes * num_samples_per_class_test]的Numpy数组，类标签的整数
    """
    random.seed(seed)
    np.random.seed(seed)
    num_samples_per_class = num_samples_per_class_train + num_samples_per_class_test
    assert num_classes <= 1623
    assert num_samples_per_class <= 20
    dim_input = 28 * 28   # 784

    # 建立文件夹
    data_folder = './omniglot_resized'
    character_folders = [os.path.join(data_folder, family, character)
                         for family in os.listdir(data_folder)
                         if os.path.isdir(os.path.join(data_folder, family))
                         for character in os.listdir(os.path.join(data_folder, family))
                         if os.path.isdir(os.path.join(data_folder, family, character))]
    random.shuffle(character_folders)
    if num_classes == -1:
        num_classes = len(character_folders)
    else:
        character_folders = character_folders[: num_classes]

    # 读取图像
    all_images = np.zeros(shape = (num_samples_per_class, num_classes, dim_input))
    all_labels = np.zeros(shape = (num_samples_per_class, num_classes))
    label_images = get_images(character_folders, list(range(num_classes)), nb_samples = num_samples_per_class, shuffle = True)
    temp_count = np.zeros(num_classes, dtype=int)
    for label,imagefile in label_images:
        temp_num = temp_count[label]
        all_images[temp_num, label, :] = image_file_to_array(imagefile, dim_input)
        all_labels[temp_num, label] = label
        temp_count[label] += 1

    # 分裂和随机排列
    train_image = all_images[:num_samples_per_class_train].reshape(-1,dim_input)
    test_image  = all_images[num_samples_per_class_train:].reshape(-1,dim_input)
    train_label = all_labels[:num_samples_per_class_train].reshape(-1)
    test_label  = all_labels[num_samples_per_class_train:].reshape(-1)
    train_image, train_label = pair_shuffle(train_image, train_label)
    test_image, test_label = pair_shuffle(test_image, test_label)
    return train_image, train_label, test_image, test_label

In [42]:
train_image, train_label, test_image, test_label = LoadData()

In [50]:
train_image.shape

(750, 784)

# 2.实现全连接神经网络

In [197]:
# 外部超参数设置
parser = argparse.ArgumentParser()
parser.add_argument('--num_classes', type=int, default=50,
                    help='number of classes used')
parser.add_argument('--num_samples_train', type=int, default=15,
                    help='number of samples per class used for training')
parser.add_argument('--num_samples_test', type=int, default=5,
                    help='number of samples per class used for testing')
parser.add_argument('--seed', type=int, default=1,
                    help='random seed')
parser.add_argument('--epochs', type=int, default=200,
                    help='number of epochs')
# args = parser.parse_args()
args = parser.parse_known_args()[0]

In [177]:
# 加载数据
train_image, train_label, test_image, test_label = LoadData(args.num_classes, args.num_samples_train, args.num_samples_test, args.seed)
# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_image), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_image), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)

In [199]:
# 加载数据的第二种思路：独热向量编码，注意后面部分代码需要修改
train_image, train_label, test_image, test_label = LoadData(args.num_classes, args.num_samples_train, args.num_samples_test, args.seed)
# 标签转换为独热向量
train_labels = np.empty((0, args.num_classes))
for i in range(0, len(train_label)):
    temp_labels = np.array([np.zeros(args.num_classes)])
    temp_labels[0, int(train_label[i])] = 1
    train_labels = np.append(train_labels, temp_labels, axis=0)
# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_image), torch.from_numpy(train_labels))
test_dataset = da.TensorDataset(torch.from_numpy(test_image), torch.from_numpy(test_label))
# 数据分批
size_batch = 50
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)

In [200]:
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
size_in = 784
size_hidden = 500
size_out = args.num_classes
learning_rate = 1e-3
# 定义模型
class ANN(nn.Module):
    def __init__(self, size_in, size_hidden, size_out):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(size_in, size_hidden)
        self.layer2 = nn.ReLU()
        self.layer3 = nn.Linear(size_hidden, size_out)
    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer2_out = self.layer2(layer1_out)
        layer3_out = self.layer3(layer2_out)
        return layer3_out
# gpu
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ANN(size_in, size_hidden, size_out)# .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [201]:
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 784)# .to(device)
        labels = labels# .to(device)
        # 前向传播
        output = model(images)
        # Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

Epoch: [1/200], Step: [1/16], Loss: 3.9054
Epoch: [1/200], Step: [2/16], Loss: 3.8809
Epoch: [1/200], Step: [3/16], Loss: 3.8884
Epoch: [1/200], Step: [4/16], Loss: 3.8904
Epoch: [1/200], Step: [5/16], Loss: 3.8835
Epoch: [1/200], Step: [6/16], Loss: 3.8802
Epoch: [1/200], Step: [7/16], Loss: 3.8147
Epoch: [1/200], Step: [8/16], Loss: 3.8476
Epoch: [1/200], Step: [9/16], Loss: 3.8158
Epoch: [1/200], Step: [10/16], Loss: 3.8720
Epoch: [1/200], Step: [11/16], Loss: 3.8220
Epoch: [1/200], Step: [12/16], Loss: 3.8076
Epoch: [1/200], Step: [13/16], Loss: 3.7825
Epoch: [1/200], Step: [14/16], Loss: 3.8097
Epoch: [1/200], Step: [15/16], Loss: 3.7676
Epoch: [2/200], Step: [1/16], Loss: 3.5201
Epoch: [2/200], Step: [2/16], Loss: 3.4793
Epoch: [2/200], Step: [3/16], Loss: 3.4065
Epoch: [2/200], Step: [4/16], Loss: 3.4104
Epoch: [2/200], Step: [5/16], Loss: 3.2338
Epoch: [2/200], Step: [6/16], Loss: 3.3448
Epoch: [2/200], Step: [7/16], Loss: 3.3032
Epoch: [2/200], Step: [8/16], Loss: 3.3165
Epoch

In [202]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 784)
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：48.8%


# 3.实现卷积神经网络CNN

In [300]:
# 外部超参数设置
parser = argparse.ArgumentParser()
parser.add_argument('--num_classes', type=int, default=50,
                    help='number of classes used')
parser.add_argument('--num_samples_train', type=int, default=15,
                    help='number of samples per class used for training')
parser.add_argument('--num_samples_test', type=int, default=5,
                    help='number of samples per class used for testing')
parser.add_argument('--seed', type=int, default=1,
                    help='random seed')
parser.add_argument('--epochs', type=int, default=200,
                    help='number of epochs')
# args = parser.parse_args()
args = parser.parse_known_args()[0]

In [301]:
# 加载数据
train_image, train_label, test_image, test_label = LoadData(args.num_classes, args.num_samples_train,
                                                            args.num_samples_test, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)

(750, 1, 28, 28)
(250, 1, 28, 28)


In [340]:
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3
# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 10, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(10, 20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(320, 100),
            torch.nn.Linear(100, 50)
        )
    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer2_out = self.layer2(layer1_out)
        layer2_out_solve = layer2_out.view(-1, 320)
        layer3_out = self.layer3(layer2_out_solve)
        return layer3_out
model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [341]:
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

Epoch: [1/200], Step: [1/8], Loss: 3.8941
Epoch: [1/200], Step: [2/8], Loss: 3.9044
Epoch: [1/200], Step: [3/8], Loss: 3.9101
Epoch: [1/200], Step: [4/8], Loss: 3.9228
Epoch: [1/200], Step: [5/8], Loss: 3.9516
Epoch: [1/200], Step: [6/8], Loss: 3.9341
Epoch: [1/200], Step: [7/8], Loss: 3.9223
Epoch: [1/200], Step: [8/8], Loss: 3.9209
Epoch: [2/200], Step: [1/8], Loss: 3.8754
Epoch: [2/200], Step: [2/8], Loss: 3.8747
Epoch: [2/200], Step: [3/8], Loss: 3.8778
Epoch: [2/200], Step: [4/8], Loss: 3.8617
Epoch: [2/200], Step: [5/8], Loss: 3.8581
Epoch: [2/200], Step: [6/8], Loss: 3.8536
Epoch: [2/200], Step: [7/8], Loss: 3.8464
Epoch: [2/200], Step: [8/8], Loss: 3.8631
Epoch: [3/200], Step: [1/8], Loss: 3.8180
Epoch: [3/200], Step: [2/8], Loss: 3.8032
Epoch: [3/200], Step: [3/8], Loss: 3.8012
Epoch: [3/200], Step: [4/8], Loss: 3.7648
Epoch: [3/200], Step: [5/8], Loss: 3.7375
Epoch: [3/200], Step: [6/8], Loss: 3.7628
Epoch: [3/200], Step: [7/8], Loss: 3.7633
Epoch: [3/200], Step: [8/8], Loss:

In [342]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：69.2%


# 4. 模型分析

### 4.1 调整类别数量

In [ ]:
num_classes = 200
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, args.num_samples_train, args.num_samples_test, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3


# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 10, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(10, 20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(320, 250),
            torch.nn.Linear(250, num_classes)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer2_out = self.layer2(layer1_out)
        layer2_out_solve = layer2_out.view(-1, 320)
        layer3_out = self.layer3(layer2_out_solve)
        return layer3_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

In [347]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：66.3%


### 4.2 调整训练/测试样本数量

In [ ]:
trian_num = 10
test_num = 10
num_classes = 100
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, trian_num,test_num, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3


# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 10, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(10, 20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(320, 200),
            torch.nn.Linear(200, num_classes)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer2_out = self.layer2(layer1_out)
        layer2_out_solve = layer2_out.view(-1, 320)
        layer3_out = self.layer3(layer2_out_solve)
        return layer3_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

In [351]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：55.5%


### 4.3不同模型结构

In [ ]:
trian_num = 18
test_num = 2
num_classes = 100
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, trian_num, test_num, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3

# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 20, kernel_size=10),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(320, 150),
            torch.nn.Linear(150, 100)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer1_out_solve = layer1_out.view(-1, 320)
        layer2_out = self.layer2(layer1_out_solve)
        return layer2_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

In [368]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：77.5%


### 4.4不同优化器

In [372]:
trian_num = 18
test_num = 2
num_classes = 100
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, trian_num, test_num, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3

# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 20, kernel_size=10),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(320, 150),
            torch.nn.Linear(150, 100)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer1_out_solve = layer1_out.view(-1, 320)
        layer2_out = self.layer2(layer1_out_solve)
        return layer2_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, args.epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

(1800, 1, 28, 28)
(200, 1, 28, 28)
Epoch: [1/200], Step: [1/19], Loss: 4.6029
Epoch: [1/200], Step: [2/19], Loss: 4.6467
Epoch: [1/200], Step: [3/19], Loss: 4.6070
Epoch: [1/200], Step: [4/19], Loss: 4.6175
Epoch: [1/200], Step: [5/19], Loss: 4.5904
Epoch: [1/200], Step: [6/19], Loss: 4.5934
Epoch: [1/200], Step: [7/19], Loss: 4.6088
Epoch: [1/200], Step: [8/19], Loss: 4.6199
Epoch: [1/200], Step: [9/19], Loss: 4.6157
Epoch: [1/200], Step: [10/19], Loss: 4.6027
Epoch: [1/200], Step: [11/19], Loss: 4.6124
Epoch: [1/200], Step: [12/19], Loss: 4.6397
Epoch: [1/200], Step: [13/19], Loss: 4.6148
Epoch: [1/200], Step: [14/19], Loss: 4.5782
Epoch: [1/200], Step: [15/19], Loss: 4.6109
Epoch: [1/200], Step: [16/19], Loss: 4.6260
Epoch: [1/200], Step: [17/19], Loss: 4.6111
Epoch: [1/200], Step: [18/19], Loss: 4.6126
Epoch: [2/200], Step: [1/19], Loss: 4.6261
Epoch: [2/200], Step: [2/19], Loss: 4.6010
Epoch: [2/200], Step: [3/19], Loss: 4.6359
Epoch: [2/200], Step: [4/19], Loss: 4.5926
Epoch: [2/

In [373]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：2.5%


### 4.5不同迭代次数

In [ ]:
epochs = 100
trian_num = 18
test_num = 2
num_classes = 100
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, trian_num, test_num, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3

# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 20, kernel_size=10),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(320, 150),
            torch.nn.Linear(150, 100)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer1_out_solve = layer1_out.view(-1, 320)
        layer2_out = self.layer2(layer1_out_solve)
        return layer2_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

In [375]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：76.0%


In [ ]:
epochs = 400
trian_num = 18
test_num = 2
num_classes = 100
# 加载数据
train_image, train_label, test_image, test_label = LoadData(num_classes, trian_num, test_num, args.seed)
# 训练用图像应该为二维，即28*28
train_images_list = list()
for i in range(0, len(train_image)):
    temp_image = np.reshape(train_image[i], (1, 28, 28))
    train_images_list.append(temp_image)
train_images = np.array(train_images_list)
print(train_images.shape)

# 测试用图像应该为二维，即28*28
test_images_list = list()
for i in range(0, len(test_image)):
    temp_image = np.reshape(test_image[i], (1, 28, 28))
    test_images_list.append(temp_image)
test_images = np.array(test_images_list)
print(test_images.shape)

# 转换为pytorch可处理数据集
train_dataset = da.TensorDataset(torch.from_numpy(train_images), torch.from_numpy(train_label))
test_dataset = da.TensorDataset(torch.from_numpy(test_images), torch.from_numpy(test_label))
# 数据分批
size_batch = 100
train_loader = da.DataLoader(dataset=train_dataset, batch_size=size_batch, shuffle=True)
test_loader = da.DataLoader(dataset=test_dataset, batch_size=size_batch, shuffle=False)
# 定义超参数, 模型, 损失函数 和 优化器
# 定义内部超参数
learning_rate = 1e-3

# 定义模型
class CNN(nn.Module):
    def __init__(self, size_out):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 20, kernel_size=10),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(320, 150),
            torch.nn.Linear(150, 100)
        )

    def forward(self, x):
        x = x.to(torch.float32)
        layer1_out = self.layer1(x)
        layer1_out_solve = layer1_out.view(-1, 320)
        layer2_out = self.layer2(layer1_out_solve)
        return layer2_out


model = CNN(size_out)  # .to(device)
# 定义损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader, 0):
        # 前向传播
        output = model(images)

        Loss = loss(output, labels.long())
        # 如果采用第二种数据加载思路，请用下面这行代码
        # Loss = loss(output, labels)
        # 反向传播
        optimizer.zero_grad()
        Loss.backward()
        optimizer.step()
        # 输出
        print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
              .format(epoch + 1, epochs, i + 1, int(len(train_image) / size_batch) + 1, Loss.item()))

In [377]:
# 模型测试
with torch.no_grad():
    total_cor = 0
    total_num = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        output = model(images)
        _, predict_result = torch.max(output.data, 1)
        total_num += labels.size(0)
        total_cor += (predict_result == labels).sum().item()
    print("Test Accuracy:：{}%".format(100 * total_cor / total_num))

Test Accuracy:：76.5%
